In [3]:
import gym
import numpy as np

env = gym.make('Taxi-v2')


In [9]:
print(env.action_space)
print(env.observation_space)
print(env.observation_space.sample())

print("Action 0:",env.step(env.action_space.sample())[0])
print("Action 1:",env.step(env.action_space.sample())[1])
print("Action 2:",env.step(env.action_space.sample())[2])
print("Action 3:",env.step(env.action_space.sample())[3])
print(env.step(env.action_space.sample()))

Discrete(6)
Discrete(500)
172
Action 0: 479
Action 1: -1
Action 2: False
Action 3: {'prob': 1.0}
(479, -1, False, {'prob': 1.0})


# AGENT

In [4]:
from collections import defaultdict

class Agent:

    def __init__(self,nA=6, epsilon=0.002, alpha=0.2, gamma=1.0):
        """ Initialize agent.

        Params
        ======
        - nA: number of actions available to the agent
        """
        self.nA = nA
        self.Q = defaultdict(lambda: np.zeros(self.nA))
        self.epsilon = epsilon
        self.alpha = alpha
        self.gamma = gamma

    def select_action(self, state, i_episode):
        """ Given the state, select an action.

        Params
        ======
        - state: the current state of the environment

        Returns
        =======
        - action: an integer, compatible with the task's action space
        """
        self.epsilon = 1.0 / ((i_episode / 800) + 1)

        policy = np.ones(self.nA) * self.epsilon / self.nA
        policy[np.argmax(self.Q[state])] = 1 - self.epsilon + self.epsilon / self.nA
        return np.random.choice(np.arange(self.nA), p=policy)
      
        #return np.random.choice(self.nA, p=policy)

    def step(self, state, action, reward, next_state, done, i_episode):
        """ Update the agent's knowledge, using the most recently sampled tuple.

        Params
        ======
        - state: the previous state of the environment
        - action: the agent's previous choice of action
        - reward: last reward received
        - next_state: the current state of the environment
        - done: whether the episode is complete (True or False)
        """
        #self.Q[state][action] += 1
        self.epsilon = 1.0 / ((i_episode / 800) + 1)

        next_policy = np.ones(self.nA) * self.epsilon / self.nA
        next_policy[np.argmax(self.Q[state])] = 1 - self.epsilon + self.epsilon / self.nA

        self.Q[state][action] = self.Q[state][action] + self.alpha * (reward + self.gamma * np.sum(self.Q[next_state] * next_policy) - self.Q[state][action])

        

# Interact

In [6]:
from collections import deque
import sys
import math
import numpy as np

def interact(env, agent, num_episodes=20000, window=100):
    """ Monitor agent's performance.
    
    Params
    ======
    - env: instance of OpenAI Gym's Taxi-v1 environment
    - agent: instance of class Agent (see Agent.py for details)
    - num_episodes: number of episodes of agent-environment interaction
    - window: number of episodes to consider when calculating average rewards

    Returns
    =======
    - avg_rewards: deque containing average rewards
    - best_avg_reward: largest value in the avg_rewards deque
    """
    # initialize average rewards
    avg_rewards = deque(maxlen=num_episodes)
    # initialize best average reward
    best_avg_reward = -math.inf
    # initialize monitor for most recent rewards
    samp_rewards = deque(maxlen=window)
    # for each episode
    for i_episode in range(1, num_episodes+1):
        # begin the episode
        state = env.reset()
        # initialize the sampled reward
        samp_reward = 0
        while True:
            # agent selects an action
            action = agent.select_action(state, i_episode)
            # agent performs the selected action
            next_state, reward, done, _ = env.step(action)
            # agent performs internal updates based on sampled experience
            agent.step(state, action, reward, next_state, done, i_episode)
            # update the sampled reward
            samp_reward += reward
            # update the state (s <- s') to next time step
            state = next_state
            if done:
                # save final sampled reward
                samp_rewards.append(samp_reward)
                break
        if (i_episode >= 100):
            # get average reward from last 100 episodes
            avg_reward = np.mean(samp_rewards)
            # append to deque
            avg_rewards.append(avg_reward)
            # update best average reward
            if avg_reward > best_avg_reward:
                best_avg_reward = avg_reward
        # monitor progress
        print("\rEpisode {}/{} || Best average reward {}".format(i_episode, num_episodes, best_avg_reward), end="")
        sys.stdout.flush()
        # check if task is solved (according to OpenAI Gym)
        if best_avg_reward >= 9.7:
            print('\nEnvironment solved in {} episodes.'.format(i_episode), end="")
            break
        if i_episode == num_episodes: print('\n')
    return avg_rewards, best_avg_reward

In [7]:
agent = Agent()
avg_rewards, best_avg_reward = interact(env, agent)

Episode 20000/20000 || Best average reward 7.5328

